# Course: Deep Learning
# Author: Sandro Camargo <sandrocamargo@unipampa.edu.br>

 Overfitting Example

 Dataset: https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.names

To open this code in your Google Colab environment, [click here](https://colab.research.google.com/github/Sandrocamargo/deep-learning/blob/master/dl_class06_overfitting.ipynb).

In [ ]:
# Import Libraries
import keras # Neural Network Library
import pandas as pd # Data Manipulation library
import numpy as np # Fast Numeric Computing library
import tensorflow as tf # Optimizers
import matplotlib.pyplot as plt # Plot library
from keras import layers # Layers to a neural network

In [ ]:
# Loading dataset
colnames=['Crim','Zn','Indus','Chas','Nox','Rm','Age','Dis','Rad','Tax','PtRatio','B','Lstat','MedV']
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data', sep=r"\s+", header=None, names=colnames)

# inspecting columns and data types from "data" dataframe
data.info()

In [ ]:
# Descriptive Statistics
data.describe()

In [ ]:
# Defining normalizing function
def normalize(dataset):
  mu = np.mean(dataset, axis = 0)
  sigma = np.std(dataset, axis = 0)
  return (dataset - mu)/sigma

data = normalize(data)

In [ ]:
# Split inputs and outputs
output = data['MedV']
inputs = data
inputs.drop(inputs.columns[[13]], axis=1, inplace=True) # column 13 is removed, because it is the output (y)

In [ ]:
np.random.seed(1) # Random numbers will be ever the same
rnd = np.random.rand(len(inputs)) < 0.8 # Training set will contain 80% of the data

# Creating the training dataset (80%)
train_x = inputs[rnd]
train_y = output[rnd]

# Creating the validation dataset (20%)
val_x = inputs[~rnd]
val_y = output[~rnd]

# Verifying dataset dimensions
print('The training dataset (inputs) dimensions are: ', train_x.shape)
print('The training dataset (outputs) dimensions are: ', train_y.shape)
print('The validation dataset (inputs) dimensions are: ', val_x.shape)
print('The validation dataset (outputs) dimensions are: ', val_y.shape)

In [ ]:
def create_model_nlayers(input_dim, num_neurons=20, num_layers=4):
    """
    Cria um modelo MLP com número customizado de camadas e neurônios.

    Args:
        input_dim (int): Número de features da entrada.
        num_neurons (int): Número de neurônios por camada oculta.
        num_layers (int): Número total de camadas ocultas.

    Returns:
        keras.Model: Modelo compilado.
    """
    # Input layer
    inputs = keras.Input(shape=(input_dim,))

    # He initialization
    initializer = tf.keras.initializers.HeNormal()

    # Primeira camada oculta
    x = layers.Dense(num_neurons, activation='relu', kernel_initializer=initializer)(inputs)

    # Camadas ocultas adicionais
    for _ in range(num_layers - 1):
        x = layers.Dense(num_neurons, activation='relu', kernel_initializer=initializer)(x)

    # Camada de saída
    outputs = layers.Dense(1)(x)

    # Modelo
    model = keras.Model(inputs=inputs, outputs=outputs, name='mlp_model')

    # Compilação
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mse']
    )

    return model


In [ ]:
# Criar modelo usando o shape do train_x
model = create_model_nlayers(input_dim=train_x.shape[1], num_neurons=20, num_layers=4)

model.summary()

In [ ]:
EPOCHS = 2000
# train model

history = model.fit(
  train_x, train_y,
  epochs = EPOCHS, verbose = 1,
  batch_size = train_x.shape[0],
  validation_data = (val_x, val_y))

# save performances
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

This plot should be generated just to inspect the learning convergence.
It is expected a decreasing of the loss function value through the epochs.

In [ ]:
plt.plot(hist['mse'], label="Training MSE")
plt.plot(hist['val_mse'], label="Validation MSE")
plt.title('Training Process')
plt.ylabel('Loss Function (MSE)')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.savefig("mlp-regr-regularization.png")
plt.show()

Similar to the previous plot, but highlighting the divergence between training and validation sets

In [ ]:
plt.plot(hist['mse'], label="Training MSE")
plt.plot(hist['val_mse'], label="Validation MSE")
plt.ylim(0,0.5)
plt.title('Training Process')
plt.ylabel('Loss Function (MSE)')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.savefig("mlp-regr-regularization-zoom.png")
plt.show()

In [ ]:
test_predictions = model.predict(val_x) # predict radon activities with the built linear regression model

plt.scatter(val_y, test_predictions, marker = 'o', c = 'blue')
plt.plot([-5,5], [-5,5], color = 'black', ls = '--')
plt.ylabel('Predictions')
plt.xlabel('Real Values')
plt.title('Regression with MLP (Testing Set)')
plt.xlim(val_y.min()*1.15, val_y.max()*1.15)
plt.ylim(test_predictions.min()*1.15, test_predictions.max()*1.15)
plt.axis(True)
plt.savefig("mlp-regr-regul-testing.png")
plt.show()

print("Correlation Coefficient in testing set: %.4f" % np.corrcoef(np.transpose(test_predictions), np.transpose(val_y))[0,1])

In [ ]:
train_predictions = model.predict(train_x) # predict radom activities with the built linear regression model

plt.scatter(train_y, train_predictions, marker = 'o', c = 'blue')
plt.plot([-5,5], [-5,5], color = 'black', ls = '--')
plt.ylabel('Predictions')
plt.xlabel('Real Values')
plt.title('Regression with MLP (Training Set)')
plt.xlim(train_y.min()*1.15, train_y.max()*1.15)
plt.ylim(train_predictions.min()*1.15, train_predictions.max()*1.15)
plt.axis(True)
plt.savefig("mlp-regr-regul-training.png")
plt.show()

print("Correlation Coefficient in training set: %.4f" % np.corrcoef(np.transpose(train_predictions), np.transpose(train_y))[0,1])